Define Generator

In [ ]:
#Based on the implementation in gan.py
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

Define Discriminator

In [ ]:
#Based on the implementation in gan.py
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

Training Loop

In [ ]:
G_losses = []
D_losses = []

for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(tqdm(dataloader)):
        
        valid = torch.ones(imgs.size(0), 1, device=device)
        fake = torch.zeros(imgs.size(0), 1, device=device)

        real_imgs = imgs.to(device)

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(imgs.size(0), latent_dim, device=device)
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        G_losses.append(g_loss.item())
        D_losses.append(d_loss.item())

        if i % sample_interval == 0:
            clear_output(wait=True)
            # print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
            # show_images(gen_imgs[:25])

            real_imgs, _ = next(iter(dataloader))
            real_imgs = denormalize(real_imgs)

            z = torch.randn(real_imgs.size(0), latent_dim).to(device)
            gen_imgs = generator(z)
            gen_imgs = denormalize(gen_imgs)

            fig, axs = plt.subplots(1, 2, figsize=(12, 6))

            axs[0].imshow(make_grid(real_imgs[:25], nrow=5).permute(1, 2, 0).cpu())
            axs[0].set_title(f'Real CelebA Faces (Epoch {epoch})')
            axs[0].axis('off')

            axs[1].imshow(make_grid(gen_imgs[:25], nrow=5).permute(1, 2, 0).detach().cpu())
            axs[1].set_title(f'Generated Faces (Epoch {epoch})')
            axs[1].axis('off')

            plt.show()


        with torch.no_grad():
            z = torch.randn(25, latent_dim).to(device)
            gen_imgs = generator(z)
            gen_imgs = denormalize(gen_imgs)
            save_image(gen_imgs, f"progress_images/epoch_{epoch:03d}.png", nrow=5, normalize=True)